In [2]:
import os 
import requests
import joblib
import base64 as b64
import pandas as pd

In [2]:
#Create spotify class which contains all functions and attributes related to spotify API
class SpotifyAPI:
    
    authorize_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_authorization_code(self):
        
        client_id = self.client_id
        client_secret = self.client_secret
        authorize_url = self.authorize_url
        client_creds = f'{client_id}:{client_secret}'
        
        client_creds_b64 = b64.b64encode(client_creds.encode())
        
        
        authorize_params = {
            'grant_type': 'client_credentials'
        }
        
        token_header = {
            'Authorization': f'Basic {client_creds_b64.decode()}' 
        }
        
        r = requests.post(authorize_url, data=authorize_params, headers=token_header)
        r_json = r.json()
        
        access_token = r_json['access_token']
        
        return access_token

In [3]:
#Load random songs csv
random_songs_df = pd.read_csv('../CSV_Data/random_songs.csv')

In [ ]:
#Set up spotify credentials
spotify_client_id = os.environ['SPOTIFY_API_CLIENT_ID']
spotify_secret_key = os.environ['SPOTIFY_API_SECRET_KEY']

spotify_client = SpotifyAPI(spotify_client_id, spotify_secret_key)

#Get access token
access_token = spotify_client.get_authorization_code()

track_header = {
    'Authorization': f'Bearer {access_token}'
}

#Get genre using spotify api
def get_genre(track):
    try:
        track_response = requests.get(f'https://api.spotify.com/v1/tracks/{track}',headers=track_header)
        track_response_json = track_response.json()

        artist_id = track_response_json['artists'][0]['id']

        artist_response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}',headers=track_header)
        artist_response_json = artist_response.json()
        genre = artist_response_json['genres']

        return genre
    except:
        print(track)

random_songs_df['genre'] = random_songs_df['id'].map(get_genre)

In [19]:
#Get format for model creation from liked_songs csv
liked_songs_df = pd.read_csv('../CSV_Data/liked_songs.csv',index_col=0)
liked_songs_df_noartists = liked_songs_df.drop(columns='artist')
model_columns = liked_songs_df_noartists.columns

In [26]:
random_songs_df = random_songs_df.rename(columns={'id':'track_id'})
random_songs_formatted = random_songs_df[model_columns]

#Format genres to only have 1 listed
random_songs_formatted['genre'] = random_songs_formatted['genre'].map(lambda x: x.split('[')[1].split(']')[0].split(', ')[0].strip("'"))
random_songs_formatted['genre'] = random_songs_formatted['genre'].map(lambda x: 'Not Available' if x == '' else x)

<ipython-input-26-6e481d44d93e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_songs_df['genre'] = random_songs_df['genre'].map(lambda x: x.split('[')[1].split(']')[0].split(', ')[0].strip("'"))
<ipython-input-26-6e481d44d93e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_songs_df['genre'] = random_songs_df['genre'].map(lambda x: 'Not Available' if x == '' else x)


In [28]:
random_songs_formatted.to_csv('../CSV_Data/random_songs_formatted.csv')